## Amazon Comprehendを利用したテキスト分析

### ライブラリのインストール

HTMLタグを除去するために利用

In [ ]:
!pip install beautifulsoup4

Amazon Comprehendのバッチ処理結果を読み取るために利用

In [ ]:
!pip install jsonlines

### データのダウンロード、確認、加工

データ格納先のバケット、Amazon Comprehendのバッチ処理実行ロールを取得

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

Amazonレビューのデータセットをダウンロード

In [ ]:
import urllib.request
import os
import gzip
import shutil

download_url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz" 
dir_name = "data"
file_name = "amazon_review.tsv.gz"
tsv_file_name = "amazon_review.tsv"
file_path = os.path.join(dir_name,file_name)
tsv_file_path = os.path.join(dir_name,tsv_file_name)

os.makedirs(dir_name, exist_ok=True)

if os.path.exists(file_path):
    print("File {} already exists. Skipped download.".format(file_name))
else:
    urllib.request.urlretrieve(download_url, file_path)
    print("File downloaded: {}".format(file_path))
    
if os.path.exists(tsv_file_path):
    print("File {} already exists. Skipped unzip.".format(tsv_file_name))
else:
    with gzip.open(file_path, mode='rb') as fin:
        with open(tsv_file_path, 'wb') as fout:
            shutil.copyfileobj(fin, fout)
            print("File uznipped: {}".format(tsv_file_path))

データの中身を確認

In [ ]:
import pandas as pd

df = pd.read_csv(tsv_file_path, sep ='\t')
df.head(5)

Amazonレビューのデータセットにはユーザーが付けたスコアが含まれているが、ここではユーザーが入力したテキスト情報を元にテキストの分析を行っていくため、必要な項目のみにデータを絞る

In [ ]:
df = df.loc[:, ['review_id', 'product_id', 'product_category', 'review_body']]

In [ ]:
df.head(20)

レビューコメントからHTMLタグを除去する

In [ ]:
from bs4 import BeautifulSoup

def filterHtmlTag(txt):
    soup = BeautifulSoup(txt, 'html.parser')
    txt = soup.get_text(strip=True)
    
    return txt

In [ ]:
# デモ用にサンプリングしてデータを小さくします
df_sample = df.sample(n=5000, random_state=42)

In [ ]:
df_sample['review_body'] = df_sample['review_body'].map(filterHtmlTag)

### Amazon ComprehendのリアルタイムAPIを利用したテキスト分析

In [ ]:
import boto3
client = boto3.client(service_name='comprehend')

サンプルのテキストを1件取得する

In [ ]:
review_text = df_sample.iloc[0,]['review_body']
review_text

エンティティ（「人」、「場所」、「位置」など）の抽出

In [ ]:
result = client.detect_entities(Text= review_text, LanguageCode='ja')
entities = result['Entities'];
for entity in entities:
    print('Entity', entity)

キーフレーズの抽出

In [ ]:
result = client.detect_key_phrases(Text= review_text, LanguageCode='ja')
keyPhrases = result['KeyPhrases'];
for keyPhrase in keyPhrases:
    print('KeyPhrase', keyPhrase)

感情の分析

In [ ]:
result = client.detect_sentiment(Text= review_text, LanguageCode='ja')
sentimentScores = result['SentimentScore'];
for sentiment in sentimentScores:
    print(sentiment, sentimentScores[sentiment])

複数のデータを一括で分析、ここでは20件まとめて分析をする

In [ ]:
df_sample_20 = df_sample[0:20]

review_text_list = []
for i, row in df_sample_20.iterrows():
    review_text = row['review_body']
    # Comprehendのの入力テキスト上限を超えないよう、ここでは1,000文字までとする
    if len(review_text) > 1000:
        review_text = review_text[0:1000]
    review_text_list.append(review_text)
    
review_text_list

分析結果を1件ずつ表示

In [ ]:
results = client.batch_detect_sentiment(TextList= review_text_list, LanguageCode='ja')
resultList = results['ResultList']
for result in resultList:
    print(result['Index'])
    sentimentScores = result['SentimentScore'];
    for sentiment in sentimentScores:
        print(sentiment, sentimentScores[sentiment])

### Amazon ComprehendのバッチAPIを利用したテキスト分析

大量のテキストデータを分析したい場合にはバッチAPIが利用可能、ここでは5,000件のデータをバッチで分析する    
バッチ分析用にテキストデータを作成
（1ファイル、1テキスト）

In [ ]:
import os

OUTPUT_DIR = './comprehend_data/'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [ ]:
review_text_list = []
for i, row in df_sample.iterrows():
    review_id = row['review_id']
    review_text = row['review_body']
    
    # Comprehendのの入力テキスト上限を超えないよう、ここでは1,000文字までとする
    if len(review_text) > 1000:
        review_text = review_text[0:1000]
    
    file_name = 'comprehend_data/' + review_id + '.txt'
    with open(file_name, mode='w') as f:
        f.write(review_text)

作成したファイル群を確認してみる

In [ ]:
!ls comprehend_data

バッチ分析対象のテキストデータの格納先S3URL、バッチ処理出力結果の格納先S3URLを作成

In [ ]:
upload_url = 's3://' + bucket + '/comprehend-exsample/batch-sentiment/'
output_url = 's3://' + bucket + '/comprehend-exsample/batch-sentiment-output/'

print(upload_url)
print(output_url)

バッチ分析対象のテキストデータをS3にアップロード

In [ ]:
!aws s3 cp comprehend_data {upload_url} --recursive

バッチ分析ジョブの実行

In [ ]:
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = 'batch-sentiment-job' + timestamp

dataAccessRoleArn = role
inputDataConfig = {"S3Uri": upload_url, "InputFormat": "ONE_DOC_PER_FILE"}
outputDataConfig = {"S3Uri": output_url}

results = client.start_sentiment_detection_job(
    DataAccessRoleArn=dataAccessRoleArn, 
    InputDataConfig=inputDataConfig,
    OutputDataConfig=outputDataConfig, 
    LanguageCode='ja'
)

In [ ]:
results

ジョブIDの取得（ステータスチェック、結果取得で利用）

In [ ]:
jobId = results['JobId']
jobId

ジョブの実行ステータスを確認
（ジョブが終了するまでポーリングでステータスを確認する。10分程度掛かります）

In [ ]:
import time

status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_sentiment_detection_job_results = client.describe_sentiment_detection_job(JobId=jobId)

    status = describe_sentiment_detection_job_results["SentimentDetectionJobProperties"]["JobStatus"]
    
    if status == "IN_PROGRESS":
        print("SentimentDetectionJob: {}".format(status))
        time.sleep(60)
    else:
        print("SentimentDetectionJob: {}".format(status))
        break


ジョブの実行結果を表示

In [ ]:
describe_sentiment_detection_job_results

分析結果の出力パスを確認

In [ ]:
describe_sentiment_detection_job_results['SentimentDetectionJobProperties']['OutputDataConfig']['S3Uri']

分析結果のデータをローカルにダウンロード

In [ ]:
!aws s3 cp {describe_sentiment_detection_job_results['SentimentDetectionJobProperties']['OutputDataConfig']['S3Uri']} .

分析結果のデータを解凍

In [ ]:
!tar -xvzf output.tar.gz

分析結果のデータを表示

In [ ]:
!head output

分析結果のデータを読み込み

In [ ]:
import jsonlines

id_list = []
sentiment_positive_list = []
sentiment_negative_list = []
sentiment_neutral_list = []
sentiment_mixed_list = []

with jsonlines.open('output') as reader:
    for obj in reader:
        review_id = obj['File'][:-4]

        id_list.append(review_id)
        
        scores = obj['SentimentScore']
        positive = scores['Positive']
        negative = scores['Negative']
        neutral = scores['Neutral']
        mixed = scores['Mixed']
        
        sentiment_positive_list.append(positive)
        sentiment_negative_list.append(negative)
        sentiment_neutral_list.append(neutral)
        sentiment_mixed_list.append(mixed)


In [ ]:
scored_df = pd.DataFrame({'review_id':id_list, 'positive':sentiment_positive_list, 'negative':sentiment_negative_list, 
                          'neutral':sentiment_neutral_list, 'mixed':sentiment_mixed_list, })

In [ ]:
scored_df.head()

元データと分析結果データをマージ

In [ ]:
joined_df = pd.merge(df_sample, scored_df, on='review_id')

In [ ]:
joined_df.head()

分析結果がネガティブだったものを確認

In [ ]:
negative_top10_df = joined_df.sort_values('negative', ascending=False)[0:10]

In [ ]:
negative_top10_df.head(10)

分析結果がポジティブだったものを確認

In [ ]:
positive_top10_df = joined_df.sort_values('positive', ascending=False)[0:10]

In [ ]:
positive_top10_df.head(10)